# Conduct PCA
- This script is used to conduct Principle Component Analysis (PCA);
- Simulations: LCZ_GM.

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import box
from shapely.geometry import Polygon
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import FunctionTransformer
from factor_analyzer import FactorAnalyzer
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [2]:
output_dir = home_path + '0_lcz_mcr/output_analysis/regional/HVI/'
factors = gpd.read_file(output_dir + 'factors/factors.shp')
factors

,name,oa21cd,LAD22NM,TS006,TS037,TS007A,TS066,TS067,TS011,TS046,TS017,TS039,TS038,duration,intensity,tbuild,pm25,geometry
0,E00030847,E00030847,Trafford,63.7,3.9,11.9,35.9,22.8,0.0,1.9,23.4,89.7,4.7,26.055556,51.622531,84.722222,6.304267,"POLYGON ((374579.881 388588.352, 374575.313 38..."
1,E00025872,E00025872,Manchester,2922.5,6.9,1.1,50.4,48.5,14.9,1.5,67.7,95.1,8.4,42.000000,53.178413,95.000000,7.525191,"POLYGON ((380953.222 389346.222, 380953 389348..."
2,E00028817,E00028817,Salford,9442.1,6.8,5.5,31.6,28.4,3.1,2.1,30.2,91.4,7.7,24.000000,50.908813,91.000000,7.955218,"POLYGON ((377435.596 401434, 377421.799 401434..."
3,E00181655,E00181655,Salford,7783.0,1.5,0.0,13.5,4.0,0.0,2.8,49.5,98.5,0.3,38.000000,53.627401,82.500000,8.755201,"POLYGON ((383502 399046, 383500 399037, 383498..."
4,E00181652,E00181652,Salford,5631.6,6.5,1.0,41.3,35.1,12.1,0.0,46.4,93.3,7.5,27.000000,51.441194,87.666667,7.559291,"POLYGON ((376601.544 397720.367, 376589.137 39..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8961,E00026032,E00026032,Manchester,5268.8,3.8,6.3,33.9,32.0,5.3,1.5,24.6,93.8,5.6,23.000000,51.704008,89.500000,8.189505,"POLYGON ((387334.868 403859.02, 387334.31 4038..."
8962,E00025701,E00025701,Manchester,7263.4,2.9,3.6,21.1,5.6,3.2,2.6,45.8,95.7,2.8,30.500000,52.362165,94.500000,7.823331,"POLYGON ((384328.975 392243.404, 384330.438 39..."
8963,E00026084,E00026084,Manchester,10708.7,1.1,3.3,13.2,7.0,0.8,2.4,29.0,95.6,0.4,25.500000,51.924526,98.000000,7.629952,"POLYGON ((381063.745 393577.351, 381066.837 39..."
8964,E00029229,E00029229,Stockport,3782.2,7.5,19.1,55.0,20.7,0.7,2.3,46.3,86.7,9.4,35.500000,52.886683,98.000000,7.637239,"POLYGON ((384459.404 388618.427, 384459.077 38..."


In [3]:
# columns = ['heat', 'pm25', 'TS006', 'TS037', 'TS007A', 'TS066', 'TS067']
# population: 'TS006'
# health: 'TS037'
# disability: 'TS038'
# unpaid care: 'TS039'
# age: 'TS007A'
# economic: 'TS066'
# qualification: 'TS067'
# deprivation: 'TS011'
# heating: 'TS046'
# household size: 'TS017'
exposure_factors = ['duration', 'intensity', 'tbuild', 'pm25', 'TS006']
sensitivity_factors = ['TS007A', 'TS017', 'TS037', 'TS038']
adaptation_factors = ['TS011', 'TS039', 'TS046', 'TS066', 'TS067']
columns = exposure_factors + sensitivity_factors + adaptation_factors
factor_data = factors[columns]
# Standardize the factors
scaler = StandardScaler()
# scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(factor_data)

# Convert back to DataFrame for easier handling
df_scaled = pd.DataFrame(data_scaled, columns=columns)
df_scaled

,duration,intensity,tbuild,pm25,TS006,TS007A,TS017,TS037,TS038,TS011,TS039,TS046,TS066,TS067
0,0.274784,0.647865,-0.043645,-2.917511,-0.981998,0.755042,-0.617198,-0.610103,-0.842786,-1.089095,-0.474737,0.282701,-0.360562,-0.623672
1,2.484866,1.487741,1.313752,-0.624173,-0.545495,-1.121739,2.907097,0.209343,-0.052772,2.059098,1.464325,0.030411,0.979277,1.561618
2,-0.010140,0.262595,0.785467,0.183575,0.449968,-0.357124,-0.076222,0.182028,-0.202234,-0.434102,0.135708,0.408846,-0.757894,-0.147500
3,1.930420,1.730107,-0.337136,1.686233,0.196644,-1.312892,1.459193,-1.265660,-1.782262,-1.089095,2.685215,0.850354,-2.430384,-2.222249
4,0.405694,0.549978,0.345231,-0.560120,-0.131849,-1.139116,1.212572,0.100083,-0.244938,1.467491,0.817971,-0.915678,0.138412,0.422206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8961,-0.148751,0.691847,0.587361,0.623651,-0.187244,-0.218104,-0.521732,-0.637418,-0.650620,0.030732,0.997514,0.030411,-0.545368,0.158611
8962,0.890834,1.047125,1.247716,-0.064158,0.117307,-0.687299,1.164838,-0.883252,-1.248469,-0.412973,1.679776,0.724209,-1.728123,-2.086200
8963,0.197777,0.810884,1.709965,-0.427392,0.643362,-0.739432,-0.171689,-1.374919,-1.760910,-0.920065,1.643868,0.598064,-2.458105,-1.967157
8964,1.583891,1.330262,1.709965,-0.413705,-0.414229,2.006228,1.204616,0.373232,0.160745,-0.941194,-1.551994,0.534991,1.404330,-0.802236


# approach 1: PCA
- ref: https://www.analyticsvidhya.com/blog/2020/10/dimensionality-reduction-using-factor-analysis-in-python/
- ref: https://www.sciencedirect.com/science/article/pii/S2212094713000054

In [4]:
df_standardized = df_scaled.copy()
pca = PCA()
pca.fit(df_standardized)
print(pca.explained_variance_ratio_)
# Calculate cumulative explained variance 累计解释方差
cumulative_explained_variance = np.cumsum(pca.explained_variance_ratio_)

# Print the cumulative explained variance to decide how many components to select
print("Cumulative explained variance:", cumulative_explained_variance)
# Set a threshold for the cumulative variance you want to retain
threshold = 0.90  # for example, 80% of the variance

# Select the number of components that explain at least the threshold percentage of the total variance
# 选择主成分数量
n_components_selected = np.argmax(cumulative_explained_variance >= threshold) + 1
print(f"Number of components selected: {n_components_selected}")

[0.30491386 0.20100938 0.14115715 0.07823438 0.05604198 0.05106702
 0.04665185 0.04053281 0.02066208 0.01865076 0.0150747  0.01157612
 0.00747235 0.00695557]
Cumulative explained variance: [0.30491386 0.50592324 0.64708039 0.72531477 0.78135674 0.83242377
 0.87907561 0.91960842 0.9402705  0.95892126 0.97399596 0.98557208
 0.99304443 1.        ]
Number of components selected: 8


In [5]:
df_explained_variance = pd.DataFrame({'cumulative_explained_variance': cumulative_explained_variance,
                                      'explained_variance': pca.explained_variance_ratio_})
df_explained_variance.to_csv('factor_variation.csv', index=False)
df_explained_variance

,cumulative_explained_variance,explained_variance
0,0.304914,0.304914
1,0.505923,0.201009
2,0.647080,0.141157
3,0.725315,0.078234
4,0.781357,0.056042
5,0.832424,0.051067
6,0.879076,0.046652
7,0.919608,0.040533
8,0.940271,0.020662
9,0.958921,0.018651


In [6]:
df_standardized = df_scaled.copy()
pca = PCA(n_components=n_components_selected)
pca.fit(df_standardized) # shape: (8966, 8)
pca_scores = pca.fit_transform(df_standardized)
# Explained variance ratio tells you the percentage of variance explained by each principal component
#explained_variance = pca.explained_variance_ratio_
# # Get principal component loadings
loadings = pca.components_

# Convert loadings to a DataFrame for easier interpretation
#loadings_df = pd.DataFrame(loadings, columns=columns, index=[f'PC{i+1}' for i in range(len(loadings))])
loadings_df = pd.DataFrame(loadings, index=[f'PC{i+1}' for i in range(loadings.shape[0])], columns=df_scaled.columns)
loadings_df

,duration,intensity,tbuild,pm25,TS006,TS007A,TS017,TS037,TS038,TS011,TS039,TS046,TS066,TS067
PC1,-0.199245,-0.180950,-0.167872,0.023034,-0.082676,0.172269,0.223813,0.421288,0.426447,0.338317,-0.197143,-0.030255,0.371064,0.401216
PC2,0.429646,0.463164,0.340018,0.309473,0.247540,-0.179868,0.238763,0.173527,0.143019,0.253786,0.210821,0.262895,0.018430,0.099948
PC3,-0.318905,-0.282284,-0.434193,0.273621,0.335625,-0.424701,0.024906,-0.065731,-0.108949,0.183140,0.277093,0.304014,-0.176490,0.107257
PC4,-0.047233,-0.085959,-0.109798,-0.224435,-0.024832,0.392328,0.609406,0.041498,0.043423,-0.230754,0.339114,0.383734,-0.023058,-0.287059
PC5,-0.028920,-0.013330,-0.031345,0.348527,0.647206,0.476521,-0.169523,-0.092267,-0.042245,-0.266323,-0.042541,-0.076365,0.336123,-0.001338
PC6,0.060322,0.011573,-0.040069,-0.463902,0.303501,-0.083714,0.035823,0.027948,-0.001188,0.193618,0.549840,-0.564264,0.137745,0.032425
PC7,0.092499,0.043612,-0.005260,-0.638013,0.327607,-0.063260,-0.270887,-0.021354,-0.025602,0.130033,-0.275153,0.523184,0.171127,0.031783
PC8,-0.012552,-0.051140,0.014989,-0.092342,0.435901,-0.142815,0.331047,0.213227,0.183920,-0.083470,-0.481810,-0.268891,-0.483256,-0.210329


In [7]:
# Perform Varimax rotation using FactorAnalyzer
# 如果加载值仍然较低，尝试进行因子旋转（如Varimax旋转），这可以帮助提高因子加载的解释性，使每个主成分能更清晰地反映出其与某些变量的关联
#Varimax旋转有时会增强主成分对特定变量的加载，从而帮助明确每个主成分的含义。
fa = FactorAnalyzer(n_factors=n_components_selected, rotation='varimax')
fa.fit(df_standardized)
rotated_loadings = fa.loadings_ # shape: (14, 8)
#loadings_df = pd.DataFrame(rotated_loadings, columns=columns, index=[f'PC{i+1}' for i in range(len(loadings))])
rotated_loadings_df = pd.DataFrame(rotated_loadings, index=df_scaled.columns, columns = [f'PC{i+1}' for i in range(rotated_loadings.shape[1])])
# Show the loadings for the first few PCs
print(rotated_loadings_df.round(2))

            PC1   PC2   PC3   PC4   PC5   PC6   PC7   PC8
duration  -0.09  0.92 -0.05  0.05  0.02  0.17 -0.01  0.02
intensity -0.04  0.93 -0.07  0.07  0.02  0.16  0.11 -0.02
tbuild    -0.10  0.82  0.03 -0.03 -0.00 -0.09  0.04  0.00
pm25       0.15  0.11 -0.12  0.11  0.04  0.25  0.65 -0.00
TS006     -0.00  0.07 -0.12  0.13 -0.03  0.54  0.22  0.06
TS007A     0.08 -0.04  0.93 -0.10  0.12 -0.27 -0.14  0.03
TS017      0.33  0.03  0.10  0.14  0.82  0.09  0.03  0.01
TS037      0.84 -0.02  0.10  0.00  0.31 -0.13  0.04  0.29
TS038      0.82 -0.03  0.18 -0.01  0.28 -0.18  0.04  0.27
TS011      0.84 -0.03 -0.25  0.07  0.12  0.11  0.06 -0.12
TS039     -0.25  0.12 -0.18  0.11  0.20  0.51  0.05 -0.15
TS046      0.01  0.05 -0.09  0.95  0.13  0.21  0.12 -0.00
TS066      0.70 -0.08  0.43 -0.05 -0.03 -0.07 -0.07 -0.10
TS067      0.85 -0.19  0.01 -0.01 -0.03 -0.08  0.20 -0.13


In [8]:
n_components_to_use = 6
select = rotated_loadings_df[rotated_loadings_df.columns[0:n_components_to_use]]
max_values = select.max(axis=1)
max_pcs = select.idxmax(axis=1)

# Combine the results into a new DataFrame for easy interpretation
result = pd.DataFrame({
    'Max_PC': max_pcs,
    'Max_Value': max_values
})

result.round(2)

,Max_PC,Max_Value
duration,PC2,0.92
intensity,PC2,0.93
tbuild,PC2,0.82
pm25,PC6,0.25
TS006,PC6,0.54
TS007A,PC3,0.93
TS017,PC5,0.82
TS037,PC1,0.84
TS038,PC1,0.82
TS011,PC1,0.84


In [9]:
#  In our case, the 5 factors together are able to explain 55.5% of the total variance.
factor_variation_df = pd.DataFrame(fa.get_factor_variance(),index=['Variance','Proportional Var','Cumulative Var'])
factor_variation_df

,0,1,2,3,4,5,6,7
Variance,3.501204,2.461265,1.245585,0.989752,0.932361,0.877829,0.576313,0.225026
Proportional Var,0.250086,0.175805,0.088970,0.070697,0.066597,0.062702,0.041165,0.016073
Cumulative Var,0.250086,0.425891,0.514861,0.585558,0.652155,0.714857,0.756022,0.772095


In [10]:
print(np.round(factor_variation_df * 100, 1))

                      0      1      2     3     4     5     6     7
Variance          350.1  246.1  124.6  99.0  93.2  87.8  57.6  22.5
Proportional Var   25.0   17.6    8.9   7.1   6.7   6.3   4.1   1.6
Cumulative Var     25.0   42.6   51.5  58.6  65.2  71.5  75.6  77.2


In [11]:
fa_reduced = FactorAnalyzer(n_factors=n_components_to_use, rotation='varimax')
fa_reduced.fit(df_standardized)
reduced_factor_loadings = fa_reduced.loadings_
#pca_scores_reduced = pca_scores[:, :n_components_to_use]
#rotated_scores = np.dot(pca_scores_reduced, reduced_factor_loadings.T)
rotated_scores = np.dot(df_standardized, reduced_factor_loadings)
df_hvi = pd.DataFrame(rotated_scores, columns=[f'HVI_{i+1}' for i in range(rotated_scores.shape[1])])
#df_hvi['Total_HVI'] = df_hvi[['HVI_1', 'HVI_2', 'HVI_3', 'HVI_4', 'HVI_5', 'HVI_6']].sum(axis=1)
df_hvi['Total_HVI'] = df_hvi.sum(axis=1)
# were organised into deciles with the result that ten HVI categories were produced
#df_hvi['HVI_Category'] = pd.qcut(df_hvi['Total_HVI'], q=10, labels=[f'Decile {i+1}' for i in range(10)])
df_hvi['HVI_Category'] = pd.qcut(df_hvi['Total_HVI'], q=10, labels=[f'{i+1}' for i in range(10)])
df_hvi.to_csv(output_dir + 'hvi_scores.csv', index=False)
df_hvi

,HVI_1,HVI_2,HVI_3,HVI_4,HVI_5,HVI_6,Total_HVI,HVI_Category
0,-3.479616,0.590117,-2.520193,0.805416,-0.757812,-0.932621,-6.294708,2
1,3.809416,4.437764,2.151332,-0.875522,2.444473,-1.015596,10.951867,10
2,-1.180740,1.071513,0.774529,-0.683425,-0.052021,0.179492,0.109348,6
3,-7.295613,4.398244,4.773695,-2.948186,0.319287,-0.719446,-1.472018,5
4,1.452567,1.064922,0.776727,-1.164826,0.748273,-0.631542,2.246121,7
...,...,...,...,...,...,...,...,...
8961,-1.672310,1.178409,1.120006,-0.827011,-0.787909,-0.132038,-1.120853,5
8962,-5.491528,3.666948,2.374561,-1.772313,0.471990,-0.805828,-1.556170,5
8963,-7.612842,3.221737,2.067137,-2.262182,-0.995866,-0.800494,-6.382509,2
8964,0.472793,3.909381,-1.529457,2.950561,1.290403,0.204643,7.298323,9
